## Intro to Web Scraping with Python


### 0. Set Up 

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup  #main webscraping package 
import requests # allows you to send https requests 
import pandas as pd # for manipulating data into data frames 
import re # regular expressions package 

Example) Jeopardy Archive Website :  https://j-archive.com/

### 1) Capture and review webpage

In [2]:
# Set Practice Link
jeopardy_archive_link = "http://www.j-archive.com/showgame.php?game_id=6623"
# Get the requested page 
page_response = requests.get(jeopardy_archive_link)

In [3]:
page_response

<Response [200]>

In [4]:
# review response 
page_response.status_code

# What do these codes mean ?  See here: https://www.restapitutorial.com/httpstatuscodes.html
# 200 = Success! 
# 400 = Client Error 

# page_response.raise_for_status()

200

In [5]:
# extract content 
page_content = BeautifulSoup(page_response.content, "html.parser")
#page_content

In [6]:
# use prettify to make output more readable 
#print(page_content.prettify())

### 2) Review main web components 

In [7]:
# Review title contents 
page_content.find_all('title')

[<title>J! Archive - Show #8215, aired 2020-05-01</title>]

In [8]:
# Also use the id feature to pull out these items 
page_content.find_all(id = 'game_title')

[<div id="game_title"><h1>Show #8215 - Friday, May 1, 2020</h1></div>]

In [9]:
# Review paragraph contents 
page_content.find_all("p")

[<p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13288" rel="external">Quemars Ahmed</a>, a field organizer from La CaÃ±ada Flintridge, California</p>,
 <p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13289" rel="external">Ashleigh McCord</a>, a marine resource management specialist from Beverly, Massachusetts</p>,
 <p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13286" rel="external">Jesse Laymon</a>, a public policy director from Long Island City, New York (whose 1-day cash winnings total $13,000)</p>,
 <p><a href="wageringcalculator.php?a=15400&amp;b=15000&amp;c=10600&amp;player_a=Jesse&amp;player_b=Quemars&amp;player_c=Ashleigh">[wagering suggestions for these scores]</a></p>]

In [10]:
# can specify on the class of the page content
page_content.find_all("p", class_ = 'contestants') 

[<p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13288" rel="external">Quemars Ahmed</a>, a field organizer from La CaÃ±ada Flintridge, California</p>,
 <p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13289" rel="external">Ashleigh McCord</a>, a marine resource management specialist from Beverly, Massachusetts</p>,
 <p class="contestants"><a href="http://www.j-archive.com/showplayer.php?player_id=13286" rel="external">Jesse Laymon</a>, a public policy director from Long Island City, New York (whose 1-day cash winnings total $13,000)</p>]

In [11]:
# Review table contents 
elems = page_content.find_all(lambda tag: tag.name == 'td')

# Another way to view this unstructured data 
#for elem in elems:
#    print(elem, end='\n'*2)


In [12]:
# just review scores 
page_content.find_all(lambda tag: tag.get('class') == ['score_positive'])

[<td class="score_positive">$5,600</td>,
 <td class="score_positive">$200</td>,
 <td class="score_positive">$9,400</td>,
 <td class="score_positive">$3,400</td>,
 <td class="score_positive">$15,400</td>,
 <td class="score_positive">$10,600</td>,
 <td class="score_positive">$15,000</td>,
 <td class="score_positive">$30,400</td>,
 <td class="score_positive">$8,800</td>,
 <td class="score_positive">$0</td>,
 <td class="score_positive">$14,400</td>,
 <td class="score_positive">$10,600</td>,
 <td class="score_positive">$13,400</td>]

### 3) Find next page link 

In [13]:
# Review hyperlinks <a> tags 
page_content.find_all(lambda tag: tag.name == 'a')[0:20]

[<a href="http://www.j-archive.com"><img alt="J! Archive" height="22" src="http://www.j-archive.com/j-a.gif" width="100"/></a>,
 <a href="http://www.j-archive.com/showseason.php?season=36">[current season]</a>,
 <a href="http://www.j-archive.com/showseason.php?season=35">[last season]</a>,
 <a href="http://www.j-archive.com/listseasons.php">[all seasons]</a>,
 <a href="http://www.j-archive.com/listprizes.php">[prizes]</a>,
 <a href="http://www.j-archive.com/wageringcalculator.php">[wagering calculator]</a>,
 <a href="http://www.j-archive.com/help.php">[help]</a>,
 <a href="showgame.php?game_id=6622">[&lt;&lt; previous game]</a>,
 <a href="http://www.j-archive.com/showplayer.php?player_id=13288" rel="external">Quemars Ahmed</a>,
 <a href="http://www.j-archive.com/showplayer.php?player_id=13289" rel="external">Ashleigh McCord</a>,
 <a href="http://www.j-archive.com/showplayer.php?player_id=13286" rel="external">Jesse Laymon</a>,
 <a href="suggestcorrection.php?clue_id=384793" rel="nofoll

In [14]:
# Can specify on the text of the page contents 
    # <a href="showgame.php?game_id=6158">[&lt;&lt; previous game]</a>
page_content.find_all(lambda tag: tag.name == 'a' and tag.text == "[<< previous game]")


[<a href="showgame.php?game_id=6622">[&lt;&lt; previous game]</a>]

In [15]:
new_game_link = page_content.find_all(lambda tag: tag.name == 'a' and 
                                                    tag.get('href') and 
                                                    tag.text == "[<< previous game]")
print(new_game_link)

[<a href="showgame.php?game_id=6622">[&lt;&lt; previous game]</a>]


In [16]:
#[<a href="showgame.php?game_id=6388">[&lt;&lt; previous game]</a>]
# WE WANT THE GAME ID ----------^^^^

# USE REGEX TO CAPTURE JUST DIGITS, and extract to a string variable [0]
re.findall(r'\d+', str(new_game_link[0]))[0] 


'6622'

In [17]:
new_game_id = re.findall(r'\d+', str(new_game_link[0]))[0]

# create next page link 
updated_jeopardy_archive_link = "http://www.j-archive.com/showgame.php?game_id=" + str(new_game_id)
updated_jeopardy_archive_link

'http://www.j-archive.com/showgame.php?game_id=6622'

### Full Web Scraping Process
Start with a recent game and move backwards through time ( Game ID 6623 OR 6389 )
http://www.j-archive.com/showgame.php?game_id=6623

In [18]:
# Initialize Variables 
index = 0
output = []
archive_link = "http://www.j-archive.com/showgame.php?game_id="
game_id = 6389
new_game_id = 0
jeopardy_archive_link = archive_link + str(game_id)

In [19]:
## Main Web Scraper  
error_catalog = []

#start extraction 
while index < 300:
    # pull page
    page_response = requests.get(jeopardy_archive_link, timeout=5) # get page, timeout set to 5 seconds 
    
    # confirm no error with pull 
    try: 
        page_response.raise_for_status()
        pass
    except:
        print("HTML ERROR CODE: " + str(page_response.status_code))
    
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    # create empty variables to fill 
    anecdotes = []
    final_scores = []
    names = []
    show_info1 = []
    show_info2 = []
    show_info3 = []
    
    #title date 
    title_date = page_content.find_all('title')[0].text # clean to just date (?)
    
    # iterate through three contestants
    for j in range(0, 3):
        #Find all anecdotes for contestants 
        paragraphs = page_content.find_all("p")[j].text
        
        # Final all final scores for contestants 
        try:
            table1 = page_content.find_all(lambda tag: tag.name == 'td' and 
                                           
                                           (tag.get('class') == ['score_positive'] 
                                            or
                                            tag.get('class') == ['score_negative'] # some scores can be negative!! See 5913
                                           )
                                          )[9:12][j].text
            pass
        
        except IndexError:
            print("Error 1 @ Game ID:" + str(new_game_id)) # error output - used to locate mismatched fields 
            error_catalog.append(jeopardy_archive_link)  # store links from archive (typically special events)
        
        #extract player names and remarks 
        try:
        table2 = page_content.find_all(lambda tag: tag.name == 'td' and 
                                   tag.get('class') == ['score_player_nickname'])[j].text
            pass
        
        except IndexError:
            print("Error 2 @ Game ID:" + str(new_game_id)) # error output - used to locate mismatched fields 
            error_catalog.append(jeopardy_archive_link)  # store links from archive (typically special events)
        
        try:
        table3 = page_content.find_all(lambda tag: tag.name == 'td' and 
                                   tag.get('class') == ['score_remarks'])[j].text
            pass
        
        except IndexError:
            print("Error 3 @ Game ID:" + str(new_game_id)) # error output - used to locate mismatched fields 
            error_catalog.append(jeopardy_archive_link)  # store links from archive (typically special events)
        
        
        #append these player details together
        anecdotes.append(paragraphs)
        final_scores.append(table1)
        names.append(table2)

    # reorder and correct data (anecdotes go 2 1 0 / names and scores go 0 1 2 )
    show_info1.extend([names[0],anecdotes[2],final_scores[0],title_date])
    show_info2.extend([names[1],anecdotes[1],final_scores[1],title_date])
    show_info3.extend([names[2],anecdotes[0],final_scores[2],title_date])
    
    #add to final output file
    output.append(show_info1)
    output.append(show_info2)
    output.append(show_info3)
    
    #create link to next page
    #create previous page number
    new_game_id = page_content.find_all(lambda tag: tag.name == 'a' and 
                                                    tag.get('href') and 
                                                    tag.text == "[<< previous game]")
        #print(new_game_id)
        #[<a href="showgame.php?game_id=6388">[&lt;&lt; previous game]</a>]
        # WE WANT THE GAME ID ----------^^^^

    new_game_id = re.findall(r'\d+', str(new_game_id[0]))[0] # USE REGEX TO CAPTURE JUST DIGITS 

    # create next page link 
    jeopardy_archive_link = archive_link + new_game_id
    jeopardy_archive_link
    
    #update iterator 
    index = index + 1 


IndentationError: expected an indented block (<ipython-input-19-b7d74b1210ca>, line 51)

In [ ]:
error_catalog

In [ ]:
#Convert output to a dataframe and prepare for initial cleaning 
archive = pd.DataFrame.from_records(output)
archive.columns = ["nickname", "player_details", "final_score", "show_info"]

# Initialize Field Name
archive["full_name"] = ""
archive["occupation"] = ""
archive["hometown"] = ""
archive["archive_info"] = ""
archive["date"] = ""

In [ ]:
archive["player_details"]

In [ ]:
# Take the archive details and split them into usable contents! 
for i in range(0,len(archive)):
    
    # extract player details 
    archive.at[i,"full_name"] = archive["player_details"].iloc[i].split(",")[0]
    archive.at[i,"occupation"] = archive["player_details"].iloc[i].split(",")[1].split("from")[0].strip()
    archive.at[i,"hometown"] = archive["player_details"].iloc[i].split("from")[1].split("(")[0].strip()
        
    # extra show info 
    archive.at[i,"archive_info"] = archive["show_info"].iloc[i].split(", aired")[0].strip()
    archive.at[i,"date"] = archive["show_info"].iloc[i].split(", aired")[1].strip()


In [ ]:
archive.head(10)

In [ ]:
# Clean date
archive['date'].replace(regex=True,inplace=True,to_replace=r':',value=r'')
archive['date'] = pd.to_datetime(archive['date'], errors='coerce')
archive.head(5)

In [ ]:
# Save your data
#archive.to_csv('../Data/jeopardy_archive_data.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path!